## TO DO:
- 

## Importing Libraries

In [1]:
import random
import numpy as np
import pandas as pd
import EagarTsai as et
import matplotlib.pyplot as plt
import seaborn as sns
import time
from rl import CustomEnv
import json
pd.set_option('display.max_rows', None)

## Initialising fixed parameters

In [2]:
"""Reading input file to get model parameter settings chosen by the user"""

f = open("model_params_train.json")
data = json.load(f)

# Assigning the values to variables so they can be used in the training and testing below

timestep = data['timestep']                 # Number of timesteps in an episode
max_steps = data['maximum_steps']           # Maximum number of timesteps across episodes in a single epoch
epochs = data['epochs']                     # Total number of epochs
model_alpha = data['model_alpha']           # Learning rate of the Bellmann equation
model_gamma = data['model_gamma']           # Discount factor of the Bellmann equation
model_epsilon = data['model_epsilon']       # Initial value for epsilon-greedy algorithm
store_data = bool(data['store_data'])       # Boolean value whether to store data locally or not

# The remaining varaibles dictate the minimum, maximum, and interval values for the three process parameters
min_power = data['min_power']
max_power = data['max_power']
interval_power = data['interval_power']
min_speed = data['min_speed']
max_speed = data['max_speed']
interval_speed = data['interval_speed']
min_hatch = data['min_hatch']
max_hatch = data['max_hatch']
interval_hatch = data['interval_hatch']

f.close()

## Creating parameter combinations

In [3]:
power = np.arange(min_power, max_power + 1, interval_power)
speed = np.arange(min_speed, max_speed + 1, interval_speed)
hatch = np.arange(min_hatch, max_hatch + 0.001, interval_hatch)

parameters = []
for p in power:
    for s in speed:
        for h in hatch:
            parameters.append((p,s,h))

In [4]:
len(parameters)

1650

## Running training model

#### The training model is where the model uses the epsilon greedy algorithm to explore the state space to find the state with the highest reward. The epsilon value comes into play and over time the agent begins exploiting more often, choosing the best action instead of a random one.

In [5]:
train_results = pd.DataFrame(columns = ['timesteps', 'test number', 'alpha', 'gamma', 'total steps', 'steps to optimal',\
                                     'optimal state', 'reward', 'reward per episode', 'time taken', 'no. of states visited'])

In [6]:
start_time = time.perf_counter()         # Beginning timer for the entire training

for j in range(0, epochs):               # For each epoch
    t1 = time.perf_counter()             # Beginning timer for the epoch
      
    # Creating instance of the environment
    env = CustomEnv(parameters, timestep, model_alpha, model_gamma, model_epsilon)
    
    episode = 0                          # Initialising episode number to 0
    states = set()                       # Total number of states visited in the epoch
    states_visited = []                  # Total number of states visited in the episode
    epoch_reward = 0                     # Total value of rewards in the epoch
    episode_rewards = []                 # Set containing each episode's total reward

    # As long as the maximum timesteps in an epoch is not exceeded
    while(env.steps < max_steps):
        env.reset(timestep)              # Resetting the environment
        done = False                     # Setting done to be False so the episode restarts
        episode += 1
        total_reward = 0                 # Episode reward initialised to zero before start of episode
        
        while not done:                  # For each episode
            reward, done = env.step(power, speed, hatch)
            total_reward += reward
            epoch_reward += reward
            states_visited.append(env.state)
            states.add(env.state)               
                
        print('States visited in episode ', episode, 'of test', j+1, 'are ', states_visited)
        
        episode_rewards.append(total_reward)
        print('------------------')
        states_visited.clear()
    
    t2 = time.perf_counter()             # Ending timer for the epoch
    time_taken = t2 - t1                 # Calculating time taken for the epoch to run
    
    row = pd.Series([timestep, j + 1, env.alpha, env.gamma, env.steps, env.optimal_steps, env.optimal_state, \
                     env.rmax, epoch_reward / episode, time_taken, len(states)], \
                    index = train_results.columns)
    train_results.loc[len(train_results)] = row
    
    # If the user opts to store the results, then for each epoch the individual step results and the overall
    # train results will be stored in separate excel files.
    if store_data:
        env.results.to_excel(f'Train Results//Epoch {j + 1} Step Results.xlsx')
        train_results.sort_values('reward', ascending = False).to_excel(f'Train Results//Epoch {j + 1} Train Results.xlsx')
        
end_time = time.perf_counter()           # Ending timer for entire training

time_elapsed = end_time - start_time     # Calculating time taken for the whole training

C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))
C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))
C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))


States visited in episode  1 of test 1 are  [359, 479, 368, 478, 489, 369, 368, 357, 236, 357, 366, 265, 266, 146, 36, 47, 46, 155, 154, 143, 42, 33, 44, 164, 53, 42, 52, 51, 52, 53, 52, 61, 172, 291, 290, 180, 300, 201, 202, 83, 182, 82, 71, 190, 70, 61, 160, 50, 40, 30, 150, 251, 151, 271, 381, 491, 381, 270, 380, 491, 391, 390, 490, 590, 471, 571, 680, 561, 672, 562, 561, 440, 441, 440, 441, 551, 550, 661, 552, 441, 551, 552, 441, 451, 341, 331, 450, 331, 220, 110, 221, 111, 1, 0, 110, 220, 121, 222, 332, 223, 122, 223, 332, 441, 330, 441, 551, 552, 452, 563, 444, 454, 455, 466, 565, 555, 664, 564, 573, 564, 565, 566, 666, 567, 457, 567, 676, 777, 887, 897, 1018, 1129, 1229, 1218, 1219, 1119, 1128, 1019, 1008, 907, 798, 688, 788, 677, 558, 458, 459, 349, 348, 228, 348, 349, 348, 358, 237, 138, 259, 368, 479, 359, 248, 239, 139, 239, 359, 478, 588, 699, 588, 589, 468, 589, 698, 597, 707, 588, 469, 369, 359, 358, 238, 137, 27, 28, 17, 126, 25, 35, 46, 55, 176, 295, 184, 285, 275, 175,

C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))


States visited in episode  2 of test 1 are  [1062, 953, 834, 843, 723, 822, 721, 822, 922, 932, 1052, 931, 941, 931, 1042, 932, 831, 720, 821, 712, 701, 582, 573, 683, 792, 683, 573, 452, 451, 332, 233, 334, 444, 564, 665, 666, 667, 666, 665, 666, 665, 676, 775, 784, 903, 1024, 1033, 934, 1035, 1135, 1134, 1034, 915, 814, 823, 942, 1041, 1040, 920, 1020, 1031, 921, 1042, 1051, 1062, 1172, 1051, 951, 851, 960, 1080, 1090, 1091, 1090, 1081, 1180, 1291, 1410, 1310, 1300, 1421, 1420, 1531, 1422, 1423, 1313, 1314, 1424, 1313, 1422, 1411, 1302, 1191, 1190, 1310, 1411, 1302, 1301, 1422, 1423, 1422, 1411, 1400, 1390, 1291, 1410, 1420, 1310, 1200, 1091, 1192, 1191, 1080, 960, 1080, 1190, 1310, 1200, 1091, 971, 1082, 971, 852, 952, 953, 1064, 943, 934, 824, 945, 834, 723, 732, 731, 732, 633, 524, 513, 502, 393, 504, 505, 624, 524, 405, 304, 413, 302, 202, 81, 70, 60, 170, 171, 62, 61, 52, 173, 293, 173, 174, 65, 184, 295, 396, 387, 396, 496, 385, 274, 394, 283, 393, 394, 403, 302, 202, 91, 210, 

OSError: Cannot save file into a non-existent directory: 'Results'

In [ ]:
print('Elapsed Time is', time_elapsed / 60, 'minutes')

In [ ]:
"""Exporting results to an Excel sheet"""

#train_results = pd.read_excel('Tugrul Results.xlsx')

In [ ]:
"""Importing results from an Excel sheet"""

#train_results.to_excel('Pre-Paper Results.xlsx', index = False)

In [ ]:
"""Displaying training details for each epoch"""

train_results.sort_values('reward', ascending = False)

In [ ]:
"""Printing best parameter configuration from each epoch"""

print("Optimal parameter configurations (P, v, h):\n")
for i in range(epochs):
    opt_state = train_results['optimal state'][i]
    print(f"Epoch {i + 1}: {parameters[int(opt_state)]}")

In [ ]:
"""Storing Q table as an Excel file if required for testing"""

# df = pd.DataFrame(data=env.qtable)

# df = (df.T)

# df.to_excel('Qtable_train.xlsx')

### Test Run

In [ ]:
"""If a specific Qtable is to be used, then import it here, otherwise the one from the train model is used"""
try:
    filename = "Qtable_train.xlsx"
    qtable = pd.read_excel(filename)
    print("Using imported Q-table")
except:
    qtable = env.qtable
    print("Using Q-table from train model")

In [ ]:
"""Reading input file to get model parameter settings chosen by the user for testing"""

f = open("model_params_test.json")
data = json.load(f)

# Assigning the values to variables so they can be used in the training and testing below

timestep = data['timestep']                 # Number of timesteps in an episode
max_steps = data['maximum_steps']           # Maximum number of timesteps across episodes in a single epoch
epochs = data['epochs']                     # Total number of epochs
model_alpha = data['model_alpha']           # Learning rate of the Bellmann equation
model_gamma = data['model_gamma']           # Discount factor of the Bellmann equation
model_epsilon = data['model_epsilon']       # Initial value for epsilon-greedy algorithm
store_data = bool(data['store_data'])       # Boolean value whether to store data locally or not

f.close()

In [ ]:
test_results = pd.DataFrame(columns = ['timestep', 'test number', 'alpha', 'gamma', 'total steps', 'steps to optimal',\
                                     'optimal state', 'reward', 'reward per episode', 'time taken', \
                                        'number of states'])

In [ ]:
# Creating a table to record only the reward for each episode
test_reward_table = pd.DataFrame(index = np.arange(1, 6, 1))
test_reward_table.index.name = 'Episode Number'

In [ ]:
start_time = time.perf_counter()         # Beginning timer for the entire training

for j in range(0,epochs):                # For each epoch
    t1 = time.perf_counter()             # Beginning timer for the epoch
    
    # Creating instance of the environment
    env_test = CustomEnv(parameters, timestep, model_alpha, model_gamma, model_epsilon, qtable = qtable)
    
    episode = 0                          # Initialising episode number to 0
    states = set()                       # Total number of states visited in the epoch
    states_visited = []                  # Total number of states visited in the episode
    epoch_reward = 0                     # Total value of rewards in the epoch
    episode_rewards = []                 # Set containing each episode's total reward

    # As long as the maximum timesteps in an epoch is not exceeded
    while(env_test.steps < max_steps):
        env_test.reset(timestep)         # Resetting the environment
        done = False                     # Setting done to be False so the episode restarts
        episode += 1                     
        total_reward = 0                 # Episode reward initialisd to zero before start of episode
        
        while not done:                  # For each episode
            reward, done = env_test.step(power, speed, hatch, test = True)
            total_reward += reward
            epoch_reward += reward
            states_visited.append(env_test.state)
            states.add(env_test.state)
        print('States visited in episode ', episode, 'of test', j+1, 'are ', states_visited)
        
        episode_rewards.append(total_reward)  
        print('------------------')
        states_visited.clear()
    
    t2 = time.perf_counter()             # Ending timer for the epoch
    time_taken = t2 - t1                 # Calculating time taken for the epoch to run
    
    test_reward_table[f'test {j + 1} rewards'] = episode_rewards
    row = pd.Series([timestep, j + 1, env_test.alpha, env_test.gamma, env_test.steps, env_test.optimal_steps,\
                     env_test.optimal_state, env_test.rmax, epoch_reward / episode, time_taken, \
                     len(states)], index = test_results.columns)
    test_results.loc[len(test_results)] = row
    
    # If the user opts to store the results, then for each epoch the individual step results and the overall
    # train results will be stored in separate excel files.
    if store_data:
        env.results.to_excel(f'Test Results//Epoch {j + 1} Step Results.xlsx')
        test_results.sort_values('reward', ascending = False).to_excel(f'Test Results//Epoch {j + 1} Test Results.xlsx')

    
end_time = time.perf_counter()           # Ending timer for entire training

time_elapsed = end_time - start_time     # Calculating time taken for the whole training

In [ ]:
print('Elapsed Time is', time_elapsed / 60, 'minutes')

In [ ]:
#test_reward_table.to_excel('Test Reward table 3.xlsx', index = False)

In [ ]:
test_results.sort_values('reward', ascending = False)

In [ ]:
"""Printing best parameter configuration from each epoch"""

print("Optimal parameter configurations (P, v, h):\n")
for i in range(epochs):
    opt_state = test_results['optimal state'][i]
    print(f"Epoch {i + 1}: {parameters[int(opt_state)]}")